In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import List, Dict, Tuple, Union, Optional
import sys
from pathlib import Path
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import leaguestandingsv3
from datetime import datetime, timezone, timedelta
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from helpfuncs import get_current_season
from pathlib import Path


pd.set_option("display.max_columns", None)


In [13]:
# Add the Backend/src/Functions path to sys.path for imports
import os
# Get current working directory and navigate to Backend/src/Functions
current_dir = Path(os.getcwd())
# Go up to project root, then to Backend/src/Functions
backend_functions_path = current_dir.parent / "Backend" / "src" / "Functions"
if str(backend_functions_path) not in sys.path:
    sys.path.insert(0, str(backend_functions_path))

# Now import the games functions
from games import *

In [14]:
eastern_conference = {
    'ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND',
    'MIA', 'MIL', 'NYK', 'ORL', 'PHI', 'TOR', 'WAS'
}

western_conference = {
    'DAL', 'DEN', 'GSW', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN',
    'NOP', 'OKC', 'PHX', 'POR', 'SAC', 'SAS', 'UTA'
}

In [15]:
class Teams():
    def __init__(self, team_abbreviation: str = None, team_id:str = None, team_name:str = None):
        self.current_season = get_current_season()
        provided = [
            team_abbreviation is not None,
            team_id is not None,
            team_name is not None
        ]

        if sum(provided) != 1:
            raise ValueError("You must provide exactly one of: team_abbreviation, team_id, team_name")

        # buscar por el parámetro correcto
        if team_abbreviation is not None:
            self.team_details = self.get_team_details_by_abbreviation(team_abbreviation)

        elif team_id is not None:
            self.team_details = self.get_team_details_by_id(team_id)

        elif team_name is not None:
            self.team_details = self.get_team_details_by_name(team_name)

        # añadir conferencia
        if not self.team_details:
            raise ValueError("Team not found with the provided identifier.")
        abb = self.team_details["abbreviation"]
        self.team_details["conference"] = "East" if abb in eastern_conference else "West"


    def check_valid_season(self, season: str) -> str:
        """Check if the provided season is valid and return formatted season."""
        if not season:
            season = self.current_season
        elif not season.isdigit():
            raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
        elif int(season) < 1980 or int(season) > datetime.now().year:
            raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
        else:
            season = f"{season}-{str(int(season)+1)[-2:]}"
        
        return season

    def get_all_teams(self) -> pd.DataFrame:
        try:
            all_teams = teams.get_teams()
            teams_df = pd.DataFrame(all_teams)
            return teams_df.to_json(orient="records")
        except Exception as e:
            print(f"Error retrieving all teams: {e}")
            raise e
    
    def get_team_details(self) -> Dict:
        try:
            return self.team_details
        except Exception as e:
            print(f"Error retrieving team details for abbreviation {self.team_abbreviation}: {e}")
            raise e

    def get_team_details_by_abbreviation(self, team_abbreviation:str) -> Dict:
        try:
            team_details = teams.find_team_by_abbreviation(team_abbreviation)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for abbreviation {team_abbreviation}: {e}")
            raise e
    
    def get_team_details_by_id(self, team_id:int) -> Dict:
        try:
            team_details = teams.find_team_name_by_id(team_id)
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for ID {team_id}: {e}")
            raise e
        
    def get_team_details_by_name(self, team_name:str) -> Dict:
        try:
            team_details = teams.find_teams_by_full_name(team_name)[0]
            return team_details
        except Exception as e:
            print(f"Error retrieving team details for name {team_name}: {e}")
            raise e

    def get_team_roster_per_season(self, season:str = None) -> pd.DataFrame:
        try:
            season = self.check_valid_season(season)

            team_id = self.team_details["id"]
            roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
            roster_data = roster.get_data_frames()[0]
            roster_data = roster_data[["PLAYER", "NUM", "POSITION", "HEIGHT", "WEIGHT", "BIRTH_DATE", "AGE", "EXP", "SCHOOL"]]
            return roster_data
        except Exception as e:
            print(f"Error retrieving roster for team {self.team_details['abbreviation']} in season {season}: {e}")
            raise e

    def get_team_last_n_games_played(self, season:str = None, last_n_games:int = 5) -> pd.DataFrame:
        season = self.check_valid_season(season)
        team_details = self.team_details
        team_id = team_details["id"]
        game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
        game_log = game_log.get_data_frames()[0]

        game_log = game_log[['Game_ID', 'GAME_DATE', 'MATCHUP', 'WL']]
        return game_log[:last_n_games]
    
    def get_team_league_standing(self, season:str = None) -> pd.DataFrame:
        season = self.check_valid_season(season)
        standings = get_current_standings(conference=self.team_details['conference']).reset_index()
        index = int(standings[standings['TeamName'] == self.team_details['nickname']].index[0]) + 1

        return index
    

    def get_team_full_info(self, season:str = None):
        pass



        
    

In [16]:
Teams('LAC').get_team_last_n_games_played()

,Game_ID,GAME_DATE,MATCHUP,WL
0,0022501205,"DEC 11, 2025",LAC @ HOU,L
1,0022500355,"DEC 06, 2025",LAC @ MIN,L
2,0022500347,"DEC 05, 2025",LAC @ MEM,L
3,0022500327,"DEC 03, 2025",LAC @ ATL,W
4,0022500313,"DEC 01, 2025",LAC @ MIA,L


In [17]:
Teams('LAC').get_team_league_standing()

13

In [18]:
#Teams().get_all_teams()
#Teams().get_team_details("LAL")
#print(Teams().get_team_roster_per_season("LAL").columns)
#Teams().get_team_roster_per_season("LAL", season=None)
#Teams("LAL").get_team_games_played_per_season(season=None, last_n_games = 5)

lakers = Teams(team_abbreviation="DEN")

#lakers.get_team_league_standing(season="2023")

team_data = lakers.get_team_details()

team_data

{'id': 1610612743,
 'full_name': 'Denver Nuggets',
 'abbreviation': 'DEN',
 'nickname': 'Nuggets',
 'city': 'Denver',
 'state': 'Colorado',
 'year_founded': 1976,
 'conference': 'West'}

In [19]:
leaderboard = Teams().get_current_standings(conference='East')

ValueError: You must provide exactly one of: team_abbreviation, team_id, team_name

In [20]:
leaderboard[leaderboard['Conference'] == 'West']

NameError: name 'leaderboard' is not defined

In [93]:
Teams('LAL').get_team_details()

{'id': 1610612747,
 'full_name': 'Los Angeles Lakers',
 'abbreviation': 'LAL',
 'nickname': 'Lakers',
 'city': 'Los Angeles',
 'state': 'California',
 'year_founded': 1948,
 'conference': 'West'}

In [9]:
eastern_conference = {
    'ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND',
    'MIA', 'MIL', 'NYK', 'ORL', 'PHI', 'TOR', 'WAS'
}

western_conference = {
    'DAL', 'DEN', 'GSW', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN',
    'NOP', 'OKC', 'PHX', 'POR', 'SAC', 'SAS', 'UTA'
}

def check_valid_season(season: str = get_current_season()) -> str:
    """Check if the provided season is valid and return formatted season."""
    if not season:
        season = get_current_season()
    elif not season.isdigit():
        raise ValueError(f"Invalid season format: {season}. Expected a year in 'YYYY' format.")
    elif int(season) < 1980 or int(season) > datetime.now().year:
        raise ValueError(f"Invalid season year: {season}. Year must be between 1980 and {datetime.now().year}.")
    else:
        season = f"{season}-{str(int(season)+1)[-2:]}"
    
    return season

def get_all_teams()-> pd.DataFrame:
    try:
        all_teams = teams.get_teams()
        teams_df = pd.DataFrame(all_teams)
        return teams_df #.to_json(orient="records")
    except Exception as e:
        print(f"Error retrieving all teams: {e}")
        raise e

def get_team_details_by_abbreviation( team_abbreviation:str) -> Dict:
    try:
        team_details = teams.find_team_by_abbreviation(team_abbreviation)
        team_details["conference"] = "East" if team_details["abbreviation"] in eastern_conference else "West"
        return team_details
    except Exception as e:
        print(f"Error retrieving team details for abbreviation {team_abbreviation}: {e}")
        raise e

def get_team_details_by_id( team_id:int) -> Dict:
    try:
        team_details = teams.find_team_name_by_id(team_id)
        team_details["conference"] = "East" if team_details["abbreviation"] in eastern_conference else "West"
        return team_details
    except Exception as e:
        print(f"Error retrieving team details for ID {team_id}: {e}")
        raise e
    
def get_team_details_by_name( team_name:str) -> Dict:
    try:
        team_details = teams.find_teams_by_full_name(team_name)[0]
        team_details["conference"] = "East" if team_details["abbreviation"] in eastern_conference else "West"
        return team_details
    except Exception as e:
        print(f"Error retrieving team details for name {team_name}: {e}")
        raise e

def get_team_roster_per_season(team_id:int = 1610612747, season:str = None) -> pd.DataFrame:
    try:
        season = check_valid_season(season)

        team_id = get_team_details_by_id(team_id)["id"]
        roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
        roster_data = roster.get_data_frames()[0]
        roster_data = roster_data[["PLAYER", "NUM", "POSITION", "HEIGHT", "WEIGHT", "BIRTH_DATE", "AGE", "EXP", "SCHOOL"]]
        return roster_data.to_json(orient='records')
    except Exception as e:
        print(f"Error retrieving roster for team {team_id} in season {season}: {e}")
        raise e

def get_team_last_n_games_played(team_id:int = 1610612747, season:str = None, last_n_games:int = 5) -> pd.DataFrame:
    season = check_valid_season(season)
    team_details = get_team_details_by_id(team_id)
    team_id = team_details["id"]
    game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    game_log = game_log.get_data_frames()[0]

    game_log = game_log[['Game_ID', 'GAME_DATE', 'MATCHUP', 'WL']]
    return game_log[:last_n_games]

def get_team_league_standing(team_id:int = 1610612747, season:str = None) -> pd.DataFrame:
    season = check_valid_season(season)
    team_details = get_team_details_by_id(team_id)
    standings = get_current_standings(conference=team_details['conference']).reset_index()
    index = int(standings[standings['TeamName'] == team_details['nickname']].index[0]) + 1

    return index


def get_team_full_info( season:str = None):
    pass



        
    

In [10]:
#get_all_teams()
get_team_roster_per_season(team_id=1610612747, season=None)

'[{"PLAYER":"Adou Thiero","NUM":"1","POSITION":"G","HEIGHT":"6-7","WEIGHT":"220","BIRTH_DATE":"MAY 08, 2004","AGE":21.0,"EXP":"R","SCHOOL":"Arkansas"},{"PLAYER":"Jarred Vanderbilt","NUM":"2","POSITION":"F","HEIGHT":"6-8","WEIGHT":"214","BIRTH_DATE":"APR 03, 1999","AGE":26.0,"EXP":"7","SCHOOL":"Kentucky"},{"PLAYER":"Dalton Knecht","NUM":"4","POSITION":"F","HEIGHT":"6-6","WEIGHT":"215","BIRTH_DATE":"APR 19, 2001","AGE":24.0,"EXP":"1","SCHOOL":"Tennessee"},{"PLAYER":"Deandre Ayton","NUM":"5","POSITION":"C","HEIGHT":"7-0","WEIGHT":"252","BIRTH_DATE":"JUL 23, 1998","AGE":27.0,"EXP":"7","SCHOOL":"Arizona"},{"PLAYER":"Gabe Vincent","NUM":"7","POSITION":"G","HEIGHT":"6-2","WEIGHT":"200","BIRTH_DATE":"JUN 14, 1996","AGE":29.0,"EXP":"6","SCHOOL":"California-Santa Barbara"},{"PLAYER":"Bronny James","NUM":"9","POSITION":"G","HEIGHT":"6-2","WEIGHT":"210","BIRTH_DATE":"OCT 06, 2004","AGE":21.0,"EXP":"1","SCHOOL":"Southern California"},{"PLAYER":"Jaxson Hayes","NUM":"11","POSITION":"C-F","HEIGHT":"7-

In [29]:
get_team_details_by_id(1610612747)

{'id': 1610612747,
 'full_name': 'Los Angeles Lakers',
 'abbreviation': 'LAL',
 'nickname': 'Lakers',
 'city': 'Los Angeles',
 'state': 'California',
 'year_founded': 1948,
 'conference': 'West'}

In [26]:
get_team_league_standing(team_id=1610612747, season=None)

4

In [ ]:
teams_df = get_all_teams()
teams_df['conference'] = teams_df['abbreviation'].apply(lambda abb: "East" if abb in eastern_conference else "West")
teams_df.drop()
teams_df

,id,full_name,abbreviation,nickname,city,state,year_founded,conference
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,East
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,East
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,East
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,West
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,East
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,West
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,West
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946,West
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967,West
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970,West
